<a href="https://colab.research.google.com/github/Murphy-Mary/MyProject/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import data

souce :https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis
!unzip /content/twitter-entity-sentiment-analysis.zip && rm /content/twitter-entity-sentiment-analysis.zip

Mounted at /content/gdrive
 50% 1.00M/1.99M [00:00<00:00, 1.99MB/s]
100% 1.99M/1.99M [00:00<00:00, 3.40MB/s]
Archive:  /content/twitter-entity-sentiment-analysis.zip
  inflating: twitter_training.csv    
  inflating: twitter_validation.csv  


In [ ]:
!pip install -q -U --pre pycaret
!python -m venv .env
!source /content/.env/bin
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda-autodetect]'
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.1/480.1 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 KB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 91.6

In [ ]:
import numpy as np
import pandas as pd
import spacy
from tqdm.auto import tqdm
import time
from sklearn.metrics import accuracy_score

In [ ]:
print(spacy.prefer_gpu())

## data

In [ ]:
df = pd.read_csv('/content/twitter_training.csv',header=None)

In [ ]:
df = df.rename(columns={0: "Id",1:"entity",2:"sentiment",3:"Tweet_content"})

In [ ]:
df

# EDA

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
 df = df[df['Tweet_content'].notna()]

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df.loc[df["sentiment"] == "Negative",'new_sent'] = 0
df.loc[df["sentiment"] == "Positive",'new_sent'] = 1
df.loc[df["sentiment"] == "Neutral",'new_sent'] = 2
df.loc[df["sentiment"] == "Irrelevant",'new_sent'] = 3
# df.loc[df["sentiment"] != "Neutral" and df["sentiment"] != "Positive" and df["sentiment"] != "Negative",'new_sent'] = 3

In [ ]:
df.isna().sum()

In [ ]:
df['new_sent'] = df['new_sent'].fillna(4)

In [ ]:
df

#prepare

In [ ]:
# Load the English NLP model
spacy.prefer_gpu() # or spacy.require_gpu()
nlp = spacy.load('en_core_web_sm')

# Define stop words
stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
def preprocess_text(text):
    # Parse the text with Spacy
    doc = nlp(text)
    
    # Lemmatize the tokens and remove stop words
    lemmas = [token.lemma_.lower() for token in doc if not token.is_stop]
    
    # Join the lemmas back into a string and return it
    return " ".join(lemmas)

In [ ]:
%%time

tqdm.pandas()

df['Tweet_content'] = df['Tweet_content'].progress_apply(preprocess_text)

In [ ]:
df

# Vectorizing the Text Data

In [ ]:
from sklearn.model_selection import train_test_split

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['Tweet_content'], df['new_sent'], test_size=0.2, random_state=248362023)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# create bag-of-words features
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# ML


## Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
# train a Naive Bayes classifier
clf = MultinomialNB()
clf

In [ ]:
clf.fit(X_train_vect, y_train)

## RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=10)
clf

In [ ]:
clf.fit(X_train_vect, y_train)

# Predictions Section

In [ ]:
# evaluate the classifier on the test set
y_pred = clf.predict(X_test_vect)

In [ ]:
y_pred

In [ ]:
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
import seaborn as sns
sns.heatmap(cm, annot=True)

# Classification Report

In [ ]:
from sklearn.metrics import classification_report

# generate classification report
target_names = ['Negative', 'Positive','Neutral','Irrelevant']
print(classification_report(y_test, y_pred, target_names=target_names))

  
NO NLP
```
              precision    recall  f1-score   support

    Negative       0.70      0.86      0.77      4297
    Positive       0.73      0.81      0.76      4009
     Neutral       0.83      0.63      0.72      3592
       other       0.81      0.62      0.70      2434

    accuracy                           0.75     14332
    macro avg       0.77      0.73      0.74     14332
    weighted avg       0.76      0.75      0.74     14332
```


```
     precision    recall  f1-score   support

    Negative       0.92      0.95      0.93      4297
    Positive       0.91      0.93      0.92      4009
     Neutral       0.91      0.92      0.92      3592
       other       0.98      0.87      0.92      2434

    accuracy                           0.92     14332
   macro avg       0.93      0.92      0.92     14332
weighted avg       0.92      0.92      0.92     14332
```


NLP


```
 precision    recall  f1-score   support

    Negative       0.68      0.85      0.76      4297
    Positive       0.71      0.78      0.75      4009
     Neutral       0.81      0.64      0.71      3592
       other       0.81      0.61      0.70      2434

    accuracy                           0.74     14332
   macro avg       0.76      0.72      0.73     14332
weighted avg       0.75      0.74      0.73     14332
```
RDF 
n = 10

```
     precision    recall  f1-score   support

    Negative       0.88      0.91      0.89      4297
    Positive       0.82      0.92      0.86      4009
     Neutral       0.92      0.85      0.88      3592
       other       0.94      0.81      0.87      2434

    accuracy                           0.88     14332
   macro avg       0.89      0.87      0.88     14332
weighted avg       0.88      0.88      0.88     14332

```
n = 100


```
  precision    recall  f1-score   support

    Negative       0.91      0.92      0.92      4297
    Positive       0.85      0.93      0.89      4009
     Neutral       0.94      0.88      0.91      3592
       other       0.96      0.85      0.90      2434

    accuracy                           0.90     14332
   macro avg       0.91      0.90      0.90     14332
weighted avg       0.91      0.90      0.90     14332
```











# validate

In [ ]:
test = pd.read_csv('/content/twitter_validation.csv',header=None)

In [ ]:
test = test.rename(columns={0: "Id",1:"entity",2:"sentiment",3:"Tweet_content"})

In [ ]:
vtest=test['new_sent']

In [ ]:
pred = clf.predict(test)

In [ ]:
pred